In [1]:
import torch

In [2]:
from model.cnn_model import Cnn_model

cnn = Cnn_model('./runs/classify/train3/')

In [3]:
from model.sam_model import Sam_model

sam = Sam_model()

In [4]:
from PIL import Image

In [5]:
def test(img_path, class_name):
    image = Image.open(img_path)
    
    rembg_image = sam.remove_bg(image)
    results = cnn.predict(rembg_image)

    target_class = cnn.get_idx(class_name=class_name)
    pred_class = cnn.get_idx(class_name=results[1])

    score = results[2]
    return target_class, pred_class, score

In [6]:
from pathlib import Path
import os
from tqdm.auto import tqdm
import time

test_path = Path('./datasets/tomato_rembg/test/')

test_results = {
    'preds' : [],
    'targets' : [],
    'scores' : []
}

for class_name in tqdm(os.listdir(test_path), desc='Class process'):
    for img_name in tqdm(os.listdir(test_path / class_name), desc='---Image predict'):
        img_path = test_path / class_name / img_name
        
        target_class, pred_class, score = test(img_path, class_name)
        
        test_results['preds'].append(pred_class)
        test_results['targets'].append(target_class)
        test_results['scores'].append(score)
    torch.cuda.empty_cache()
    # time.sleep(30)
        # break

Class process:   0%|          | 0/10 [00:00<?, ?it/s]

---Image predict:   0%|          | 0/100 [00:00<?, ?it/s]

---Image predict:   0%|          | 0/100 [00:00<?, ?it/s]

---Image predict:   0%|          | 0/100 [00:00<?, ?it/s]

---Image predict:   0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from model.utils import plot_confmat
import pandas as pd 

df = pd.DataFrame(test_results)
plot_confmat([], df, threshold=.9)